In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#loading training dataset
train=pd.read_csv("train.csv",index_col="PassengerId")
#loading testdataset
test=pd.read_csv("test.csv",index_col="PassengerId")


In [2]:
#checking train dataset
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#Name and ticket are both unique for each passenger thus they will not be much effective for training our model
#drop useless values
test=test.drop(["Name","Ticket"],axis=1)
train=train.drop(["Name","Ticket"],axis=1)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 69.6+ KB


In [5]:
#contains 8 independent features and 1 target or dependent feature i.e Survived 
#total 891 rows
#3 object type , 4 int type and 2 float type features
#lets get some more insight
train.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
#lets check for null values in the features
train.isnull().sum().sort_values(ascending=False)

Cabin       687
Age         177
Embarked      2
Fare          0
Parch         0
SibSp         0
Sex           0
Pclass        0
Survived      0
dtype: int64

In [7]:
#cabin ,Age and Embarked contains null values in train dataset
#lets check for test dataset
test.isnull().sum().sort_values(ascending=False)

Cabin       327
Age          86
Fare          1
Embarked      0
Parch         0
SibSp         0
Sex           0
Pclass        0
dtype: int64

In [8]:
#as null values of embarked are not present in test dataset thus we can drop Embarked null rows
#dropping embarked with null values
drop_rows=train[train["Embarked"].isnull()==True].index.tolist()
train=train.drop(drop_rows,axis=0)
#now lets verify
train.isnull().sum().sort_values(ascending=False)

Cabin       687
Age         177
Embarked      0
Fare          0
Parch         0
SibSp         0
Sex           0
Pclass        0
Survived      0
dtype: int64

In [9]:
#lets analyse cabin column
train["Cabin"].head()

PassengerId
1     NaN
2     C85
3     NaN
4    C123
5     NaN
Name: Cabin, dtype: object

In [10]:
#position of the cabin can be useful in analysing survival rate thus first letter needs to be extracted
#thus transforming Cabin column into categorical feature
#lets fill the null values in Cabin with X to represent another category in Cabin
train["Cabin"]=train["Cabin"].fillna("X")
test["Cabin"]=test["Cabin"].fillna("X")
#now extracting 1st letter from each Cabin value to make it as categorical feature
for i in range(len(train["Cabin"])):
    train["Cabin"].iloc[i]=train["Cabin"].iloc[i][0]
for i in range(len(test["Cabin"])):
    test["Cabin"].iloc[i]=test["Cabin"].iloc[i][0]

F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
train.isnull().sum().sort_values(ascending=False)

Age         177
Embarked      0
Cabin         0
Fare          0
Parch         0
SibSp         0
Sex           0
Pclass        0
Survived      0
dtype: int64

In [12]:
test.isnull().sum().sort_values(ascending=False)

Age         86
Fare         1
Embarked     0
Cabin        0
Parch        0
SibSp        0
Sex          0
Pclass       0
dtype: int64

In [13]:
#Alright now , lets take care of the null values in Age feature 
train["Age"]=train["Age"].fillna(train["Age"].mean())
test["Age"]=test["Age"].fillna(test["Age"].mean())

In [14]:
#now the only remaining null value is in Fare
test["Fare"]=test["Fare"].fillna(test["Fare"]).mean()

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 1 to 891
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    object 
 3   Age       889 non-null    float64
 4   SibSp     889 non-null    int64  
 5   Parch     889 non-null    int64  
 6   Fare      889 non-null    float64
 7   Cabin     889 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 69.5+ KB


Survived=Target feature
Pclass,Sex,Cabin and Embarked=Categorical columns

In [16]:
#and now all the null values and useless feature has been taken care of
#Seperating dependent and independent dataset
#from train dataframe
X=train.iloc[:,1:].values
Y=train.iloc[:,0].values
#from test dataframe
test_X=test.iloc[:,:].values

In [17]:
#Handling categorical Data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[("encoder",OneHotEncoder(),[0,1,6,7])],remainder="passthrough")
X=np.array(ct.fit_transform(X))
test_X=np.array(ct.transform(test_X))

In [18]:
#no of categories in each categorical columns 
print(len(train.Pclass.unique()),len(train.Sex.unique()),len(train.Cabin.unique()),len(train.Embarked.unique()))

3 2 9 3


In [19]:
#therefore we need to discard 0,3,5 and 14 (one from each category) will remove dummy variable trap
X=np.delete(X,[0,3,5,14],axis=1)
test_X=np.delete(test_X,[0,3,5,14],axis=1)

In [20]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
Classifier=RandomForestClassifier(n_estimators=100)
Classifier.fit(X,Y)

#checking accuracy
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=Classifier,X=X,y=Y,cv=10,n_jobs=-1)
print(accuracies.mean())
print(accuracies.std())

0.8099336057201226
0.04322558970405298


In [21]:
#this shows that data is in category of high accuracy and high variance
#lets try grid search to improve accuracy by getting optimal hyperparameter

In [22]:
from sklearn.model_selection import GridSearchCV
#parameters contains options to be selected as optimal
parameters={"max_depth":[5,10,15,20],
            "min_samples_leaf":[1,2,3],
            "min_samples_split":[2,5,8,11],
            "n_estimators":[90,100,110],
            "random_state":[0,10,20]}
gridsearch=GridSearchCV(estimator=Classifier,param_grid=parameters,scoring="accuracy",cv=10,n_jobs=-1)
gridsearch=gridsearch.fit(X,Y)
print(gridsearch.best_params_)


{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 90, 'random_state': 20}


In [23]:
print(gridsearch.best_score_)

0.8380362614913176


In [24]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
Classifier=RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=5,n_estimators=90,random_state=20)
Classifier.fit(X,Y)

from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=Classifier,X=X,y=Y,cv=10)
print(accuracies.mean())
print(accuracies.std())



0.8380362614913176
0.052941026023721846


In [27]:
tPred=Classifier.predict(test_X)

PassengerId=np.array(test.index)
Survived=np.array(tPred)

combined = np.vstack((PassengerId,Survived)).T
df=pd.DataFrame(combined,columns=["PassengerId","Survived"])

df.to_csv("predicted.csv",index=False)